In [2]:
import requests, json
from ai_api_client_sdk.ai_api_v2_client import AIAPIV2Client

In [5]:
# Please replace the configurations below.
# config_id: The target configuration to create the deployment. Please create the configuration first.
with open("config.json") as f:
    config = json.load(f)

with open("env.json") as f:
    env = json.load(f)

deployment_id = env["deployment_id"]
resource_group = config.get("resource_group", "default")
print("deployment id: ", deployment_id, " resource group: ", resource_group)

deployment id:    resource group:  


#### Initiate connection to SAP AI Core


In [8]:
aic_sk = config["ai_core_service_key"]
base_url = aic_sk["serviceurls"]["AI_API_URL"] + "/v2/lm"
ai_api_client = AIAPIV2Client(
    base_url= base_url,
    auth_url=aic_sk["url"] + "/oauth/token",
    client_id=aic_sk['clientid'],
    client_secret=aic_sk['clientsecret'],
    resource_group=resource_group)

KeyError: 'ai_core_service_key'

In [9]:
token = ai_api_client.rest_client.get_token()
headers = {
        "Authorization": token,
        'ai-resource-group': resource_group,
        "Content-Type": "application/json"}

NameError: name 'ai_api_client' is not defined

#### Check the deployment status

In [10]:
# Check deployment status before inference request
deployment_url = f"{base_url}/deployments/{deployment_id}"
response = requests.get(url=deployment_url, headers=headers)
resp = response.json()    
status = resp['status']

deployment_log_url = f"{base_url}/deployments/{deployment_id}/logs"
if status == "RUNNING":
        print(f"Deployment-{deployment_id} is running. Ready for inference request")
else:
        print(f"Deployment-{deployment_id} status: {status}. Not yet ready for inference request")
        #retrieve deployment logs
        #{{apiurl}}/v2/lm/deployments/{{deploymentid}}/logs.

        response = requests.get(deployment_log_url, headers=headers)
        print('Deployment Logs:\n', response.text)

NameError: name 'base_url' is not defined

#### Pull the model into Ollama

In [11]:
model = "nomic-embed-text"
#model = "llama3:8b"
#model = "mistral:latest"
#model = "phi3:latest" 
#model = "mistral:7b-instruct-q5_K_M"
#model = "mixtral:8x7b-instruct-v0.1-q4_0" #Important: please resource plan to infer.l in byom-oss-llm-templates/ollama-template.yaml

deployment = ai_api_client.deployment.get(deployment_id)
inference_base_url = f"{deployment.deployment_url}/v1"
openai_base_url = deployment.deployment_url

NameError: name 'ai_api_client' is not defined

In [ ]:
# pull the model from ollama model repository
endpoint = f"{inference_base_url}/api/pull"
print(endpoint)

#let's pull the mistral model from ollama
json_data = {  "name": model}

response = requests.post(endpoint, headers=headers, json=json_data)
print('Result:', response.text)